# Введение в методы сбора и обработки данных
## Добашнее задание к уроку 2

# Данное домашнее задание еще не готово, и размещено чтобы не вышло время сдачи. В ближейшее время доработаю.

In [34]:
import requests
import pprint
from collections import Counter
import re

### Задание 1
Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

#### Решение:

Реализуем функцию которая переводит названия городов в их IATA

In [4]:
def get_city_iata_by_name(name1: str, name2: str):
    """Возвращает IATA городов, принимая на вход их названия"""
    
    link = "https://www.travelpayouts.com/widgets_suggest_params"
    string = f'Из {name1} в {name2}'
    
    req = requests.get(link, params={'q': string})
    data = req.json()
    
    return data['origin']['iata'], data['destination']['iata']

# Проверим что функия отрабатыает корректно
get_city_iata_by_name('архангельск', 'Красноярск')

('ARH', 'KJA')

Реализуем функцию которая выдает список цен за билет в зависимости от указанных городов отправления и назначения:

In [22]:
def get_min_price(city1: str, city2: str):
    link = "http://min-prices.aviasales.ru/calendar_preload"
    fromCity, toCity = get_city_iata_by_name(city1, city2)
    params = {
        'origin': fromCity,
        'destination': toCity
    }
    
    req = requests.get(link, params=params)
    data = req.json()
    
    print(f'Результат поиска авиабилетов из {city1.capitalize()} в {city2.capitalize()}:')
    for i in data['best_prices']:
        print(f"Вылет: {i['depart_date']}, Цена: {i['value']:.0f}руб.")
    
        
#get_min_price('архангельск', 'Красноярск')

Наконец, попросим пользователя ввести города для поиска билетов:

In [27]:
city1 = input("Укажите пункт отправления: ")
city2 = input("Укажите пункт назначения:  ")
print()

get_min_price(city1.capitalize(), city2.capitalize())

Укажите пункт отправления: Москва
Укажите пункт назначения:  Екатеринбург

Результат поиска авиабилетов из Москва в Екатеринбург:
Вылет: 2020-06-05, Цена: 11432руб.
Вылет: 2020-03-17, Цена: 7200руб.
Вылет: 2019-12-15, Цена: 3002руб.
Вылет: 2019-11-11, Цена: 3002руб.
Вылет: 2019-12-06, Цена: 3998руб.
Вылет: 2020-01-12, Цена: 6200руб.
Вылет: 2020-01-17, Цена: 6200руб.
Вылет: 2019-11-19, Цена: 4800руб.
Вылет: 2019-10-31, Цена: 3502руб.
Вылет: 2019-09-08, Цена: 7700руб.
Вылет: 2019-12-04, Цена: 4800руб.
Вылет: 2019-12-08, Цена: 6200руб.
Вылет: 2020-02-19, Цена: 6900руб.
Вылет: 2020-06-26, Цена: 11432руб.
Вылет: 2019-07-27, Цена: 9766руб.
Вылет: 2019-09-05, Цена: 7200руб.
Вылет: 2020-01-08, Цена: 11432руб.
Вылет: 2019-12-24, Цена: 6900руб.
Вылет: 2019-09-13, Цена: 7203руб.
Вылет: 2019-12-28, Цена: 9800руб.
Вылет: 2019-08-19, Цена: 6425руб.
Вылет: 2019-08-11, Цена: 8200руб.
Вылет: 2020-07-14, Цена: 11714руб.
Вылет: 2019-08-24, Цена: 8200руб.
Вылет: 2019-10-15, Цена: 3002руб.
Вылет: 2019-12-1

### Задание 2
В приложении парсинга википедии получить первую ссылку на другую страницу и вывести все значимые слова из неё. Результат записать в файл в форматированном виде

#### Решение:

In [56]:
def return_wiki_html(topic):
    """Возвращает html-код страницы"""
    wiki_request = requests.get(f'https://ru.wikipedia.org/wiki/{topic.capitalize()}')
    return wiki_request.text


def return_words(topic):
    """Возвращает 10 самых употребляемых на странице слов"""
    wiki_html = return_wiki_html(topic)
    words = re.findall('[а-яА-Я]{3,}', wiki_html)
    words_counter = Counter(words)
    
    return words_counter.most_common(10)


def find_first_link(topic):
    """Выводит первую ссылку со страницы"""
    text = return_wiki_html(topic)
    ans = re.search("href\s*=\"(http\S*)\"", text)
    return ans.group(1)

find_first_link('Кролик')

'https://ru.wikipedia.org/wiki/%D0%9A%D1%80%D0%BE%D0%BB%D0%B8%D0%BA%D0%B8'

Пройдем по первой ссылке на искомой странице, и выведем оттуда в файл 10 наиболее часто используемых слов:

In [57]:
page_name = 'Государство'

# Найдем первую ссылку с данной странице
first_link = find_first_link(page_name)

link_most_common_words = return_words(first_link)

In [65]:
with open('first_link_most_common_words', 'w') as f:
    f.write(f'Список наиболее часто употребляемых слов на странице {first_link}:\n')
    f.writelines([f'{l[0]} - {l[1]} раз.\n' for l in link_most_common_words])

In [66]:
!cat first_link_most_common_words

Список наиболее часто употребляемых слов на странице https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%81%D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2%D0%BE:
Государство - 11 раз.
Википедия - 6 раз.
Википедии - 6 раз.
Перейти - 6 раз.
страницы - 6 раз.
Поиск - 5 раз.
страницу - 5 раз.
Список - 5 раз.
для - 4 раз.
можно - 4 раз.


### Задание 3
Научить приложение определять количество ссылок в статье. Спарсить каждую ссылку и результаты записать в отдельные файлы.